In [36]:
from typing import Dict, List, Any
import json

In [37]:
game_state = {
    "world":{
        "locations": {
            "loc_Havenwood": {
                "name": "Havenwood",
                "type": "Town",
                "description": "A small, resilient town built amidst the colossal ruins of the past. Wooden structures are nestled against ancient, vine-covered Elven stonework.",
                "pointsOfInterest": {
                "loc_WearyWandererTavern": {
                    "name": "The Weary Wanderer Tavern",
                    "description": "The heart of Havenwood. The air is thick with the smell of woodsmoke, roasting meat, and spilled ale. A large, crackling fireplace dominates one wall."
                }
                },
                "connections": {
                "north_path": "loc_Gloomwood"
                },
                "state": ["peaceful", "tense_undercurrent"]
            },
            "loc_Gloomwood": {
                "name": "The Gloomwood",
                "type": "Forest",
                "description": "A dark and menacing forest of ancient ironwood trees. Sunlight struggles to pierce the thick canopy, and strange sounds echo from the shadows.",
                "connections": {
                "south_path": "loc_Havenwood"
                },
                "state": ["dangerous", "unnatural_silence"]
            }
        }
    },
    "actors": {
        "pcs": {
            "pc_Elara": {
                "name": "Elara",
                "class": "Rogue",
                "race": "Human",
                "background": "Outlander",
                "stats": {
                "hp_current": 22,
                "hp_max": 22,
                "ac": 14,
                "str": 10,
                "dex": 17,
                "con": 12,
                "int": 14,
                "wis": 11,
                "cha": 13
                },
                "inventory": ["item_Shortsword", "item_LeatherArmor", "item_ThievesTools", "item_MapFragment"],
                "statusEffects": [],
                "relationships": {
                "npc_BoricTheBarkeep": "friendly",
                "fact_IronwoodSentinels": "neutral"
                },
                "personalGoals": ["Discover the meaning of the symbols on the strange map fragment."]
            }
        },
        "npcs": {
            "npc_GuardCaptainThorne": {
                "name": "Guard Captain Thorne",
                "role": "Guard",
                "currentLocation": "loc_Havenwood",
                "dispositionToParty": "neutral",
                "stats": { "hp_current": 30, "hp_max": 30, "ac": 16 },
                "knowledge": [
                    {
                        "id": "know_HavenwoodHistory",
                        "info": "Havenwood was once a thriving Elven settlement before the Great War.",
                        "revealed": False
                    }
                ],
                "dialogue_state": {
                    "mood": "cautious"
                }
            },
            "npc_BoricTheBarkeep": {
                "name": "Boric the Barkeep",
                "role": "Quest Giver / Merchant",
                "currentLocation": "loc_WearyWandererTavern",
                "dispositionToParty": "friendly",
                "stats": { "hp_current": 18, "hp_max": 18 },
                "knowledge": [
                {
                    "id": "know_GloomwoodDangers",
                    "info": "The Gloomwood has become more aggressive lately; the wolf packs are getting bolder.",
                    "revealed": False
                }
                ],
                "dialogue_state": {
                "mood": "worried"
                }
            },
            "npc_GloomfangWolf": {
                "name": "Gloomfang Wolf",
                "role": "Hostile Creature",
                "currentLocation": "loc_Gloomwood",
                "dispositionToParty": "hostile",
                "stats": { "hp_current": 11, "hp_max": 11, "ac": 13 }
            }
        }
    },
    "journal": "",
    "session": {
        "currentLocation": "loc_Havenwood"
    },
    "history": ""
}

In [38]:
def update_session_by_location(game_state: Dict) -> Dict:
    """
    Update the session information based on the current location.
    """
    current_session = game_state.get("session", {})
    current_location_id = current_session.get("currentLocation")

    if not current_location_id:
        return {"error": "No current location set in session."}
    
    location_details: Dict = None
    locations = game_state.get("world", {}).get("locations", {})
    if current_location_id in locations:
        location_details = locations[current_location_id]
    else:
        return {"error": f"Location ID '{current_location_id}' not found in world locations."}
    
    actor_details = {"pcs": [], "npcs": []}
    pcs = game_state.get("actors", {}).get("pcs", {})
    for pc_id, pc in pcs.items():
        if pc.get("currentLocation") == current_location_id:
            if actor_details is None:
                actor_details["pcs"].append(pc)

    npcs = game_state.get("actors", {}).get("npcs", {})
    for npc_id, npc in npcs.items():
        if npc.get("currentLocation") == current_location_id:
            actor_details["npcs"].append(npc)

    session_update = {
        "currentLocation": {
            current_location_id: location_details
        },
        "currentActors": {
            "pcs": actor_details["pcs"],
            "npcs": actor_details["npcs"]
        }
    }

    return session_update

In [39]:
session = update_session_by_location(game_state)
game_state["session"] = session
print(json.dumps(game_state["session"], indent=4))


{
    "currentLocation": {
        "loc_Havenwood": {
            "name": "Havenwood",
            "type": "Town",
            "description": "A small, resilient town built amidst the colossal ruins of the past. Wooden structures are nestled against ancient, vine-covered Elven stonework.",
            "pointsOfInterest": {
                "loc_WearyWandererTavern": {
                    "name": "The Weary Wanderer Tavern",
                    "description": "The heart of Havenwood. The air is thick with the smell of woodsmoke, roasting meat, and spilled ale. A large, crackling fireplace dominates one wall."
                }
            },
            "connections": {
                "north_path": "loc_Gloomwood"
            },
            "state": [
                "peaceful",
                "tense_undercurrent"
            ]
        }
    },
    "currentActors": {
        "pcs": [],
        "npcs": [
            {
                "name": "Guard Captain Thorne",
                "role